# Compare Hail VEP output VS Docker VEP output.
## Step 1: Convert Hail Table to flattened TSV:

In [1]:
import hail as hl
import pandas as pd
hl.stop()
hl.init()

file="data/vep-hail-out/genebass.hailvep.gnomad_process_csqs.chr21.ht"

vep = hl.read_table(file)
vep = vep.annotate(worst_csq_by_gene_canonical = vep.vep.worst_csq_by_gene_canonical)
vep = vep.drop('vep')
vep = vep.explode('worst_csq_by_gene_canonical')
vep = vep.flatten()
vep.export(f"data/vep-hail-out/genebass.hailvep.gnomad_process_csqs.chr21.tsv")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/gpfs3/well/lindgren/users/hjo721/conda/skylake/envs/popgen/lib/python3.7/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://compa023.hpc.in.bmrc.ox.ac.uk:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.115-10932c754edb
LOGGING: writing to /gpfs3/well/lindgren/barney/brava-ground-truth-annotations/hail-20

## Step 2: Import both TSV's - Basic Comparisons:

In [14]:
import pandas as pd

# Load the files into pandas dataframes
hail_vep = pd.read_csv('data/vep-hail-out/genebass.hailvep.gnomad_process_csqs.chr21.tsv', sep='\t')
docker_vep = pd.read_csv('data/docker-vep-out/genebass.chr21.worst_csq_by_gene_canonical.txt', sep=' ')

# Construct the equivalent SNP_ID for hail_vep by combining locus and alleles columns
hail_vep['SNP_ID'] = hail_vep['locus'] + ':' + hail_vep['alleles'].str.replace('[\[\]" ]', '').str.replace(',', ':')
docker_vep['SNP_ID'] = 'chr' + docker_vep['SNP_ID']

# Extract relevant columns
hail_vep_subset = hail_vep[['SNP_ID', 'worst_csq_by_gene_canonical.gene_id']]
docker_vep_subset = docker_vep[['SNP_ID', 'GENE']]

# Merge dataframes on SNP_ID/locus and GENE/gene_id to find the overlap
overlap = pd.merge(hail_vep_subset, docker_vep_subset, left_on=['SNP_ID', 'worst_csq_by_gene_canonical.gene_id'], 
                   right_on=['SNP_ID', 'GENE'], how='inner')

print(f"Number of overlapping entries between Hail VEP and Docker VEP: {len(overlap)}")
print(f"Number of unique entries in Hail VEP: {hail_vep_subset.drop_duplicates().shape[0]}")
print(f"Number of unique entries in Docker VEP: {docker_vep_subset.drop_duplicates().shape[0]}")

# Check for entries in Hail VEP not present in Docker VEP
missing_in_docker = hail_vep_subset.loc[~hail_vep_subset.set_index(['SNP_ID', 'worst_csq_by_gene_canonical.gene_id']).index.isin(overlap.set_index(['SNP_ID', 'worst_csq_by_gene_canonical.gene_id']).index)]
print(f"Entries in Hail VEP not present in Docker VEP: {len(missing_in_docker)}")

# Check for entries in Docker VEP not present in Hail VEP
missing_in_hail = docker_vep_subset.loc[~docker_vep_subset.set_index(['SNP_ID', 'GENE']).index.isin(overlap.set_index(['SNP_ID', 'GENE']).index)]
print(f"Entries in Docker VEP not present in Hail VEP: {len(missing_in_hail)}")


/well/lindgren/users/hjo721/conda/skylake/envs/popgen/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


Number of overlapping entries between Hail VEP and Docker VEP: 48867
Number of unique entries in Hail VEP: 117024
Number of unique entries in Docker VEP: 49068
Entries in Hail VEP not present in Docker VEP: 68157
Entries in Docker VEP not present in Hail VEP: 201
